In [19]:
import pandas as pd
import numpy as np

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import faiss

from datasets import load_dataset

from pprint import pprint

from tqdm import tqdm

import logging
logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt

import psycopg2

len_dataset = 2326839

In [2]:
dataset = load_dataset('json', data_files='../GeneratingEmbeddings/dataset/arxiv_data.json', split='train')

In [3]:
next(iter(dataset)).keys()

dict_keys(['id', 'submitter', 'authors', 'title', 'comments', 'journal-ref', 'doi', 'report-no', 'categories', 'license', 'abstract', 'versions', 'update_date', 'authors_parsed'])

In [4]:
def connect_to_db():
    conn = psycopg2.connect(
        host="localhost",
        port=5432,
        database="vector_database",
        user="postgres",
        password="admin"
    )
    conn.autocommit = True

    cur = conn.cursor()
    cur.execute("CREATE EXTENSION IF NOT EXISTS vector")
    return cur, conn

cur, conn = connect_to_db()

In [13]:
cur.execute("select embedding from article_embeddings where article_id = 0;")

res = cur.fetchone()[0]

In [14]:
res

'[-0.4060021,0.02999023,0.34609616,-0.27283677,0.07611577,-0.16192625,-0.449993,0.031084234,0.122885756,-0.017351806,0.05440324,-0.2626167,-0.4044795,0.3559492,-0.1033407,0.7818314,0.2092371,-0.08259787,-0.053812437,0.06575841,0.7928784,0.0940029,-0.22114345,0.70127785,0.024205253,-0.2249983,0.1275823,0.01782413,0.15365149,0.16776991,-0.17156973,0.6440639,-0.111504346,-0.29046157,0.5019062,0.32385027,-0.19542378,-0.19713552,0.25302288,0.19405997,-0.5531399,-0.5443914,0.028747512,-0.06782242,-0.38825893,-0.26412058,0.28860793,-0.20195928,0.00014734175,-0.31094173,-0.6896056,-0.44595724,0.22710237,0.42306954,0.008430531,0.70128703,-0.061496746,-0.0908522,0.2183619,0.020819416,-0.39582044,-0.21404684,-0.05325823,0.29468346,0.30809534,0.06826572,0.35505128,0.25191793,-0.45497802,0.22067167,-0.26044866,0.35225424,-0.19611177,0.16263436,-0.12024753,0.15612757,0.022235783,0.45075762,-0.1652342,-0.083497465,-0.28259605,0.16425751,-0.18618381,-0.18807416,-0.18623647,0.16131827,-0.09086518,0.131

In [15]:
cur.execute("select * from article_embeddings where article_id != 0 order by embedding <=> %s::vector limit 10000;", (res,))

In [16]:
pprint(len(cur.fetchall()))

10000


In [14]:
pprint(dataset[2160760]['abstract'])   

('  The gold-plated discovery mode of a Standard Model like Higgs boson at '
 'the\n'
 'CERN Large Hadron Collider (LHC) is the H -> Z^0 Z^0 decay mode. To find '
 'and\n'
 'then measure the properties of the Higgs, it is crucial to have the most\n'
 'precise theoretical prediction both for the signal and the QCD background '
 'in\n'
 'this mode. In this work we calculate the effects of the initial-state '
 'multiple\n'
 'soft-gluon emission on the kinematic distributions of photon and Z^0 pairs\n'
 'produced in hadron collisions. The Collins-Soper-Sterman formalism is '
 'extended\n'
 'to resum the large logarithmic terms due to soft-gluons. The resummed total\n'
 'rates, the invariant mass, transverse momentum, and rapidity distributions '
 'of\n'
 'the photon and Z^0 pairs, and the transverse momentum distributions of the\n'
 'individual vector bosons are presented and compared to the fixed order\n'
 'predictions in the whole kinematic range, for the LHC energies and for the\n'
 'up

In [16]:
pprint(dataset[21152]['abstract'])

('  M. E. Cesaro (1885) gave a quite remarkable expression for the Bell '
 'number\n'
 '--the number of partitions of an n-element set -- as a definite integral. '
 'This\n'
 'note is an exposition, correcting a typographical error in the original.\n')


In [17]:
pprint(dataset[90480]['abstract'])

('  Martin Klazar computed the total weight of ordered trees under 12 '
 'different\n'
 'notions of weight. The last and perhaps most interesting of these weights,\n'
 'w_{12}, led to a recurrence relation and an identity for which he requested\n'
 'combinatorial explanations. Here we provide such explanations. To do so, we\n'
 'introduce the notion of a "Klazar violator" vertex in an increasing ordered\n'
 'tree and observe that w_{12} counts what we call Klazar trees--increasing\n'
 'ordered trees with no Klazar violators. A highlight of the paper is a '
 'bijection\n'
 'from n-edge increasing ordered trees to perfect matchings of '
 '[2n]={1,2,...,2n}\n'
 'that sends Klazar violators to even numbers matched to a larger odd number. '
 'We\n'
 'find the distribution of the latter matches and, in particular, establish '
 'the\n'
 'one-summation explicit formula sum_{k=1}^{lfloor n/2 rfloor}(2k-1)!!^2\n'
 'StirlingPartition{n+1}{2k+1} for the number of perfect matchings of [2n] '
 'with

In [2]:
'''
Interesting Queries

SELECT
    a.id AS article1_id,
    b.id AS article2_id,
    (1 - (a.embedding <=> b.embedding)) AS similarity
FROM
    David_Callan_Papers a
    JOIN David_Callan_Papers b ON a.id < b.id
ORDER BY similarity;

Time: 3 seconds when run on a view containing 54 rows

'''


'\nInteresting Queries\n\nSELECT\n    a.id AS article1_id,\n    b.id AS article2_id,\n    (1 - (a.embedding <=> b.embedding)) AS similarity\nFROM\n    David_Callan_Papers a\n    JOIN David_Callan_Papers b ON a.id < b.id\nORDER BY similarity;\n\nTime: 3 seconds when run on a view containing 54 rows\n\n'

In [14]:
def get_similar_articles_base(article_id, num_articles=3):
    cur.execute("select embedding from bert_embeddings where article_id = %s;", (article_id,))
    res = cur.fetchone()[0]
    cur.execute("select * from bert_embeddings where article_id != %s order by embedding <=> %s::vector limit %s;", (article_id, res, num_articles))
    return cur.fetchall()

def get_similar_articles_tiny(article_id, num_articles=3):
    cur.execute("select embedding from bert_tiny_embeddings where article_id = %s;", (article_id,))
    res = cur.fetchone()[0]
    cur.execute("select * from bert_tiny_embeddings where article_id != %s order by embedding <=> %s::vector limit %s;", (article_id, res, num_articles))
    return cur.fetchall()

def get_article_metadata(article_id):
    cur.execute("select * from paper_metadata where id = %s;", (article_id,))
    return cur.fetchone()

In [15]:
get_similar_articles_base(0)

[(2160760,
  '[-0.4488568,0.11970043,0.5010775,-0.28573996,0.13609388,-0.09250909,-0.32589304,0.06667821,0.13345182,-0.113251224,0.14393815,-0.1318554,-0.26465943,0.3631766,-0.039968986,0.66497326,0.4008696,-0.1780251,-0.027300423,0.013995059,1.0079768,0.02360821,-0.08993359,0.7900914,0.13410643,-0.10306542,0.1981546,0.18495762,0.12115329,0.12408829,-0.044892445,0.5010305,-0.23560557,-0.24779569,0.4728099,0.41068697,-0.2869312,-0.21333426,0.2627789,0.3726291,-0.3239389,-0.64510804,-0.08921166,-0.15524735,-0.44479805,-0.26846945,0.13366395,-0.4872489,-0.025237571,-0.539935,-0.6384919,-0.30519253,0.17578292,0.29105514,0.06769079,0.53944564,-0.02912383,-0.17952067,0.22148696,-0.036187235,-0.46785793,-0.23472017,0.045718968,0.2688585,0.4098596,-0.056775495,0.39749062,0.17407988,-0.5491408,0.22210538,-0.38707575,0.39230454,-0.13865665,0.2672279,-0.121719465,0.14979538,0.10802026,0.5279015,-0.13349244,-0.12360644,-0.2429582,0.11485115,-0.16070199,-0.19222878,-0.21926668,0.27487716,0.03659426

In [12]:
pprint(get_article_metadata(0))
pprint(dataset[0]['abstract'])
pprint(get_article_metadata(2160760))
pprint(dataset[2160760]['abstract'])

(0,
 'Calculation of prompt diphoton production cross sections at Tevatron and\n'
 '  LHC energies',
 "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan",
 'hep-ph')
('  A fully differential calculation in perturbative quantum chromodynamics '
 'is\n'
 'presented for the production of massive photon pairs at hadron colliders. '
 'All\n'
 'next-to-leading order perturbative contributions from quark-antiquark,\n'
 'gluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\n'
 'all-orders resummation of initial-state gluon radiation valid at\n'
 'next-to-next-to-leading logarithmic accuracy. The region of phase space is\n'
 'specified in which the calculation is most reliable. Good agreement is\n'
 'demonstrated with data from the Fermilab Tevatron, and predictions are made '
 'for\n'
 'more detailed tests with CDF and DO data. Predictions are shown for\n'
 'distributions of diphoton pairs produced at the energy of the Large Hadron\n'
 'Collider (LHC). Distributions of

In [16]:
get_similar_articles_tiny(0)

[(112449,
  '[-0.38298827,0.38121578,0.13359398,-0.2543156,0.44394773,-0.11095295,-0.26802254,-0.8979639,-1.3673751,1.1398691,0.088314034,0.2264219,0.3411805,-0.3051958,0.25812203,-0.5514767,-0.76301426,-0.30396605,-0.5286335,0.45165667,0.33239472,-0.8850682,0.019215673,1.2642858,2.2622168,0.38091052,0.32828522,0.55267525,1.0461779,0.10227744,-0.7721046,-0.8593786,0.34954745,-1.3632433,-0.14433855,-0.8507621,0.17736867,-0.7099898,-0.07070906,-0.5147346,0.5842942,-0.03816922,0.36535683,-1.3201518,0.43483797,-0.06564162,-0.35416055,0.68498135,0.23876837,-0.30778295,0.053189956,1.9279855,0.23487273,-0.95408255,0.2801985,-0.27142003,-0.13511667,0.054194167,-0.09239517,1.5199142,0.99998593,-0.6671233,-1.2436829,-1.0187246,-0.51303196,0.7796854,-0.08698118,-0.11516079,-0.76185644,-0.028595772,-0.65602636,-0.24800737,0.08960439,-1.4003437,-0.6636828,0.009938402,0.41943797,-0.04122547,2.449645,0.21240097,-0.14518394,-0.5496225,-0.6183314,0.87712103,1.1871108,-0.99629927,0.0033014491,1.4456269,

In [21]:
get_similar_articles_base(0, 100000);

In [20]:
get_similar_articles_tiny(0, 100000);

In [5]:
def load_all_embeddings():
    e_list = []
    for i in range(1, 19):
        embeddings = np.load(f'../GeneratingEmbeddings/bert_embeddings/embeddings_{i}000.npz')['arr_0'].reshape(-1, 768)
        e_list.append(embeddings)
    
    e = np.concatenate(e_list, axis=0)
    return e

embeddings_array = load_all_embeddings()

In [8]:
# load the index

index = faiss.read_index("../GeneratingEmbeddings/Indexes/bert_embeddings.index")

In [22]:
# using the index, find the 3 most similar vectors to the first one in the embeddings array

D, I = index.search(embeddings_array[0:1], 2304000)

In [24]:
I.shape

(1, 2304000)

<!-- make a markdown table with the columns query, time-->

| Query        | Time (seconds)     | FAISS   |
| --------     | -------- | --------|
| `SELECT AVG(embedding) FROM bert_embeddings;` | 11.0 |
| `SELECT AVG(embedding) FROM bert_tiny_embeddings;` | 2.7 |
| `SELECT * FROM bert_embeddings WHERE article_id != %s ORDER BY embedding <=> %s::vector LIMIT 3;` | 1.8 | 0.5
| `SELECT * FROM bert_tiny_embeddings WHERE article_id != %s ORDER BY embedding <=> %s::vector LIMIT 3;` | 0.2 |
| `SELECT * FROM bert_embeddings WHERE article_id != %s ORDER BY embedding <=> %s::vector LIMIT 100000;` | 5.4 | 0.4
| `SELECT * FROM bert_tiny_embeddings WHERE article_id != %s ORDER BY embedding <=> %s::vector LIMIT 100000;` | 1.4 |
| `SELECT * FROM bert_embeddings WHERE article_id != %s ORDER BY embedding <=> %s::vector LIMIT 100000;` | Not able to run | 0.6


In [29]:
# create a view in the database that contains the contains all of the paper metadata and the embeddings for the author David Callan, use the tabels paper_metadata and bert_embeddings
cur.execute("CREATE OR REPLACE VIEW david_callan_papers AS SELECT * FROM paper_metadata JOIN bert_embeddings ON paper_metadata.id = bert_embeddings.article_id WHERE paper_metadata.authors LIKE '%David Callan%';")